In [69]:
# 문제 1
import pandas as pd

df = pd.read_csv("drinks.csv")
# print(df)
# print(df.head())
# df.info()
# df['continent'].value_counts()

# 대륙별 맥주 소비량의 평균
mean_val = df.groupby("continent")['beer_servings'].mean()
# print(mean_val)

# 평균이 가장 큰 대륙
continent_top = mean_val.idxmax()
print(continent_top)

# 평균이 가장 큰 대륙 중 맥주 소비량이 5번째 많은 국가
result = (df['continent'] == continent_top)
# df = df[result]
# print(df)
df_new = df[result].sort_values('beer_servings', ascending=False)
print(df_new)
print(df_new.iloc[4, 1])
print(df_new.iloc[4][['beer_servings', 'country']])

Europe
                country  beer_servings  spirit_servings  wine_servings  \
45       Czech Republic            361              170            134   
65              Germany            346              117            175   
135              Poland            343              215             56   
98            Lithuania            343              244             56   
81              Ireland            313              118            165   
140             Romania            297              122            167   
16              Belgium            295               84            212   
160               Spain            284              157            112   
151              Serbia            283              131            127   
93               Latvia            281              216             62   
9               Austria            279               75            191   
156            Slovenia            270               51            276   
60              Finland        

In [15]:
import pandas as pd

df = pd.read_csv("drinks.csv", usecols=["beer_servings", "wine_servings", "total_litres_of_pure_alcohol", "continent"])
df.head()

df.groupby("continent")['beer_servings'].mean()

continent
Africa            61.471698
Asia              37.045455
Europe           193.777778
North America    145.434783
Oceania           89.687500
South America    175.083333
Name: beer_servings, dtype: float64

In [87]:
# 문제2
import pandas as pd

df = pd.read_csv("tourist.csv")
df

df['방문객 합계'] = df['관광'] + df['공무'] + df['사업'] + df['기타']
df['관광객비율'] = df['관광'] / df['방문객 합계']
# print(df)

# 방법1
# 관광객 비율이 두 번째로 높은 국가
# print(df.sort_values('관광객비율', ascending=False))
# a = df.sort_values('관광객비율', ascending=False).iloc[1, 3]
# print(a)
# print(df.sort_values('관광', ascending=False))
# b = df.sort_values('관광', ascending=False).iloc[1, 2]
# print(b)
# print(a + b)

# 방법2
# a = df.nlargest(2, '관광객비율').iloc[1, 3]
a = df.nlargest(2, '관광객비율').iloc[1]['사업']
print(a)
b = df.nlargest(2, '관광').iloc[1, 2]
print(b)
print(a + b)

203
238
441


In [93]:
# 문제3
import pandas as pd

df = pd.read_csv("chem.csv")
print(df)
print(df.info())

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df['co'] = scaler.fit_transform(df[['co']])
df['nmhc'] = scaler.fit_transform(df[['nmhc']])

print(df)

# 표준편차
co_std = df['co'].std()
print(co_std)
nmhc_std = df['nmhc'].std()
print(nmhc_std)

result = round((co_std - nmhc_std), 3)
print(result)

   sample   co  nmhc  etc
0     샘플1   79    54   31
1     샘플2   84    57   58
2     샘플3  109    74  113
3     샘플4   15    77   21
4     샘플5   65    77  115
..    ...  ...   ...  ...
95   샘플96   56    58   86
96   샘플97   33    59  111
97   샘플98   55    79  105
98   샘플99   69    51   59
99  샘플100   94    45   59

[100 rows x 4 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   sample  100 non-null    object
 1   co      100 non-null    int64 
 2   nmhc    100 non-null    int64 
 3   etc     100 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 3.3+ KB
None
   sample        co      nmhc  etc
0     샘플1  0.680851  0.500000   31
1     샘플2  0.734043  0.534091   58
2     샘플3  1.000000  0.727273  113
3     샘플4  0.000000  0.761364   21
4     샘플5  0.531915  0.761364  115
..    ...       ...       ...  ...
95   샘플96  0.436170  0.545455   86
96   샘플97 

#### 제2유형

In [33]:
import pandas as pd

train = pd.read_csv("churn_train.csv")
test = pd.read_csv("churn_test.csv")

# 데이터 탐색 
# print(train.head())
# train.info()

# print(train.describe(include="object"))
# print(test.describe(include="object"))

# customerID - 삭제
train = train.drop("customerID", axis=1)
test = test.drop("customerID", axis=1)

# object 칼럼 인코딩 
cols = train.select_dtypes(include='object').columns
# print(cols)

# 레이블 인코딩
from sklearn.preprocessing import LabelEncoder, StandardScaler
encoder = LabelEncoder()
for col in cols:
    train[col] = encoder.fit_transform(train[col])
    test[col] = encoder.fit_transform(test[col])

# train.info()

# 변수 선택
X = train.drop("TotalCharges", axis=1)  # 독립(설명) 변수
y = train["TotalCharges"] # 종속(예측) 변수

# 설명 변수 데이터 정규화(스케일링)
# X = StandardScaler().fit(X).transform(X)

# 학습용(훈련/검증) 데이터 분할
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                y, test_size=0.2, random_state=30)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# 모델 학습 및 검증 
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
# 랜덤 포레스트
# model = RandomForestRegressor(random_state=0)
# 선형 회귀
model = LinearRegression()

model.fit(X_train, y_train)
pred = model.predict(X_test)
# print(pred)

mae = mean_absolute_error(y_test, pred)
print("MAE:", mae)

# 평가용 데이터 - 예측 및 파일 제출
pred2 = model.predict(test)
print(pred2)

result = pd.DataFrame({'pred': pred2})
result.to_csv("result.csv", index=False)

pd.read_csv("result.csv")

(3292, 17) (824, 17) (3292,) (824,)
MAE: 905.0304116302589
[3169.6108  845.     3810.9618 ... 4111.0562 1001.5328 3822.9305]


,pred
0,3169.6108
1,845.0000
2,3810.9618
3,985.7980
4,1388.9680
...,...
1759,316.2095
1760,3844.0138
1761,4111.0562
1762,1001.5328


#### 제3유형

In [143]:
# 문제1
import pandas as pd
from statsmodels.formula.api import logit
df = pd.read_csv("churn.csv")
# print(df.head())
df.info()

# 종속변수 - Churn, 독립변수 - 나머지 칼럼
formula = """
    Churn ~ AccountWeeks + ContractRenewal + DataPlan + DataUsage + CustServCalls +
    DayMins + DayCalls + MonthlyCharge + OverageFee + RoamMins
"""

model = logit(formula, data=df).fit()

# 유의하지 않은 독립변수의 개수 구하기
# print(model.summary())
# Intercept항 제외
print(model.pvalues[1:])

print(model.pvalues[1:] > 0.05)
# True의 합
print(sum(model.pvalues[1:] > 0.05))

# 문제2
# 유의한 변수 선택
print(model.pvalues < 0.05)

# 로지스틱 회귀
model2 = logit('Churn ~ DataUsage + DayMins', data=df).fit()
print(model2.summary())
# 회귀 계수의 합계
print("회귀계수:", model2.params)
print("회귀계수의 합계:",round(sum(model2.params), 3))

import numpy as np
# 'DataUsage'의 회귀 계수
coef = model2.params['DataUsage']
print(coef)

# 오즈비 계산 - 5만큼 증가
print(round(np.exp(coef*5), 3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Churn            1000 non-null   int64  
 1   AccountWeeks     1000 non-null   int64  
 2   ContractRenewal  1000 non-null   int64  
 3   DataPlan         1000 non-null   int64  
 4   DataUsage        1000 non-null   float64
 5   CustServCalls    1000 non-null   int64  
 6   DayMins          1000 non-null   float64
 7   DayCalls         1000 non-null   int64  
 8   MonthlyCharge    1000 non-null   float64
 9   OverageFee       1000 non-null   float64
 10  RoamMins         1000 non-null   float64
dtypes: float64(5), int64(6)
memory usage: 86.1 KB
Optimization terminated successfully.
         Current function value: 0.393603
         Iterations 6
AccountWeeks       0.265370
ContractRenewal    0.621969
DataPlan           0.145825
DataUsage          0.019118
CustServCalls      0.063182
Day

In [151]:
# 문제 2
import pandas as pd
df = pd.read_csv("piq.csv")
print(df.head())

from statsmodels.formula.api import ols
model = ols("PIQ ~ Brain + Height + Weight", data=df).fit()
print(model.summary())

# p-value가 가장 작은 변수의 회귀 계수
coef = model.params['Brain']
print("회귀 계수:", round(coef, 3))

# 결정 계수 - 모델이 종속변수의 변동을 얼마나 잘 설명하는지 나타내는 지표이다.
print("결정 계수:", round(model.rsquared, 2))

# 새로운 데이터 생성
new_df = pd.DataFrame({
    'Brain': [90],
    'Height': [70],
    'Weight': [150]
})
new_df

# PIQ 예측
pred = model.predict(new_df)
print(pred)
print(type(pred)) # 시리즈
print("PIQ:", round(pred[0])) #인덱스로 찾기

   PIQ  Brain  Height  Weight
0  132  85.78    62.5     127
1   96  86.54    68.0     135
2   84  90.49    66.3     134
3  134  79.06    62.0     122
4   86  88.91    70.0     180
                            OLS Regression Results                            
Dep. Variable:                    PIQ   R-squared:                       0.370
Model:                            OLS   Adj. R-squared:                  0.329
Method:                 Least Squares   F-statistic:                     9.024
Date:                Fri, 24 Jan 2025   Prob (F-statistic):           8.25e-05
Time:                        11:51:42   Log-Likelihood:                -217.27
No. Observations:                  50   AIC:                             442.5
Df Residuals:                      46   BIC:                             450.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std e